On reddit someone had asked the question: "[D] the more features we have, the more accurate is a decision tree (always)?" and there were a few educated guesses as to the outcome on training data (assuming for a fixed depth tree). The goal of this notebook is to determine if training accuracy will improve or decrease as more features are added. The secondary objective is to see what change in accuracy one could recieve on validation given the same circumstances.

I chose the Cat in the dat challenge dataset as it is sufficiently large to believe the results should generalize well and because I could reuse the data prepairing code I already wrote from when I tried this competition.


Original thread: https://www.reddit.com/r/MachineLearning/comments/lfj0r5/d_the_more_features_we_have_the_more_accurate_is/

The first codeblock handles the data preprocessing, the cell was hidden to keep the notebook length down, feel free to unhide it and look it over should you choose. Basically, all ordinal features were label encoded based on order, all non ordinal categorical features were randomly label encoded (tree based algorithms can make sence of these features so long as cardinality isn't extreme), and the hexidecial values were returned to base 10.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 225)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels

print('Setup Complete')

In [ ]:
file_path = '../input/cat-in-the-dat/train.csv'
train_raw = pd.read_csv(file_path, index_col='id')

file_path = '../input/cat-in-the-dat/test.csv'
test_raw = pd.read_csv(file_path, index_col='id')

z = train_raw.ord_5.unique()

feature_select = 'month'

q = train_raw[feature_select].unique()
r = test_raw[feature_select].unique()
q_not_r = []
r_not_q = []
both = []
t = 0

for val in q:
    if val in r:
        both = both + [val]
    else:
        q_not_r = q_not_r + [val]

for val in r:
    if val in q:
        t = t + 1
    else:
        r_not_q = r_not_q + [val]

q = train_raw.ord_5.unique()
q = sorted(q)
five = {}
x = 0
for val in q:
    five[val] = x
    x = x + 1
#print(five)

def ord_five_a(val):
    return five.get(val)



q = train_raw.ord_3.unique()
q = sorted(q)
three = {}
x = 0
for val in q:
    three[val] = x
    x = x + 1
def ord_three_a(val):
    return three.get(val)




q = train_raw.ord_4.unique()
q = sorted(q)
four = {}
x = 0
for val in q:
    four[val] = x
    x = x + 1
    
def ord_four_a(val):
    return four.get(val)

features = ['bin_0', 'bin_1', 'bin_3', 'bin_4', 'nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5','nom_6', 'nom_7', 'nom_8', 'nom_9', 
            'ord_0', 'ord_1', 'ord_2','ord_3', 'ord_4', 'ord_5', 'day', 'month']

def hexx(val):
    return int(val, 16)

def ord_five(val):
    return ord_alph(val[0])

def ord_fiveish(val):
    return ord_alph(val[1])

def ord_alph(val):
#    if val.dtype == "int":
#        return val
    val = val.lower()
    if val == 'a':
        return 1
    if val == 'b':
        return 2
    if val == 'c':
        return 3
    if val == 'd':
        return 4
    if val == 'e':
        return 5
    if val == 'f':
        return 6
    if val == 'g':
        return 7
    if val == 'h':
        return 8
    if val == 'i':
        return 9
    if val == 'j':
        return 10
    if val == 'k':
        return 11
    if val == 'l':
        return 12
    if val == 'm':
        return 13
    if val == 'n':
        return 14
    if val == 'o':
        return 15
    if val == 'p':
        return 16
    if val == 'q':
        return 17
    if val == 'r':
        return 18
    if val == 's':
        return 19
    if val == 't':
        return 20
    if val == 'u':
        return 21
    if val == 'v':
        return 22
    if val == 'w':
        return 23
    if val == 'x':
        return 24
    if val == 'y':
        return 25
    if val == 'z':
        return 26
    else:
        return 0
    
def ord_five_b(val):
    if val[0] == val[0].lower():
        val_0 = ord_alph(val[0]) + 26
    else:
        val_0 = ord_alph(val[0])
    if val[1] == val[1].lower():
        val_1 = ord_alph(val[1]) + 26
    else:
        val_1 = ord_alph(val[1])
    val_1 = val_1/100
    return(val_0 + val_1)

def ord_two(val):
    if val == 'Cold':
        return 2
    if val == 'Hot':
        return 4
    if val == 'Lava Hot':
        return 6
    if val == 'Boiling Hot':
        return 5
    if val == 'Freezing':
        return 1
    if val == 'Warm':
        return 3
    else:
        return 0

def ord_one(val):
    if val == 'Grandmaster':
        return 1
    if val == 'Expert':
        return 3
    if val == 'Novice':
        return 5
    if val == 'Contributor': 
        return 4
    if val == 'Master':
        return 2
    else:
        return 0
    
def nom_four(val):
    if val == 'Bassoon':
        return 1
    if val == 'Piano':
        return 2
    if val == 'Theremin':
        return 3
    if val == 'Oboe':
        return 4
    else:
        return 0
    
def nom_three(val):
    if val == 'Finland':
        return 1
    if val == 'Russia':
        return 2
    if val == 'Canada':
        return 3
    if val == 'Costa Rica':
        return 4
    if val == 'China':
        return 5
    if val == 'India':
        return 6
    else:
        return 0
    
def nom_two(val):
    if val == 'Snake':
        return 1
    if val == 'Hamster':
        return 2
    if val == 'Lion':
        return 3
    if val == 'Cat':
        return 4
    if val == 'Dog':
        return 5
    if val == 'Axolotl':
        return 6
    else:
        return 0
    
def nom_one(val):
    if val == 'Triangle':
        return 1
    if val == 'Trapezoid':
        return 2
    if val == 'Polygon':
        return 3
    if val == 'Square':
        return 4
    if val == 'Star':
        return 5
    if val == 'Circle':
        return 6
    else:
        return 0
    
def nom_zero(val):
    if val == 'Green' or val == 0:
        return 0
    if val == 'Blue' or val == 1:
        return 1
    if val == 'Red' or val == 2:
        return 2
    else:
        return 3
    
def bin_three(val):
    if val == 'T' or val == 1:
        return 1
    if val == 'F' or val == 2:
        return 2
    else:
        return 0
    
def bin_four(val):
    if val == 'y' or val == 1:
        return 1
    if val == 'N' or val == 2:
        return 2
    else:
        return 0
    
train_raw['nom_0'] = pd.Series([nom_zero(x) for x in train_raw.nom_0], index=train_raw.index)
train_raw['nom_1'] = pd.Series([nom_one(x) for x in train_raw.nom_1], index=train_raw.index)
train_raw['nom_2'] = pd.Series([nom_two(x) for x in train_raw.nom_2], index=train_raw.index)
train_raw['nom_3'] = pd.Series([nom_three(x) for x in train_raw.nom_3], index=train_raw.index)
train_raw['nom_4'] = pd.Series([nom_four(x) for x in train_raw.nom_4], index=train_raw.index)
train_raw['bin_3'] = pd.Series([bin_three(x) for x in train_raw.bin_3], index=train_raw.index)
train_raw['bin_4'] = pd.Series([bin_four(x) for x in train_raw.bin_4], index=train_raw.index)
train_raw['ord_1'] = pd.Series([ord_one(x) for x in train_raw.ord_1], index=train_raw.index)
train_raw['ord_2'] = pd.Series([ord_two(x) for x in train_raw.ord_2], index=train_raw.index)
train_raw['ord_3'] = pd.Series([ord_alph(x) for x in train_raw.ord_3], index=train_raw.index)
train_raw['ord_4'] = pd.Series([ord_alph(x) for x in train_raw.ord_4], index=train_raw.index)
#train_raw['ord_5_plus'] = pd.Series([x for x in train_raw.ord_5], index=train_raw.index)
#train_raw['ord_5_plus'] = pd.Series([ord_alph(x[1]) for x in train_raw.ord_5_plus], index=train_raw.index)
train_raw['ord_5'] = pd.Series([ord_five_b(x) for x in train_raw.ord_5], index=train_raw.index)
train_raw['nom_5'] = pd.Series([hexx(x) for x in train_raw.nom_5], index=train_raw.index)
train_raw['nom_6'] = pd.Series([hexx(x) for x in train_raw.nom_6], index=train_raw.index)
train_raw['nom_7'] = pd.Series([hexx(x) for x in train_raw.nom_7], index=train_raw.index)
train_raw['nom_8'] = pd.Series([hexx(x) for x in train_raw.nom_8], index=train_raw.index)
train_raw['nom_9'] = pd.Series([hexx(x) for x in train_raw.nom_9], index=train_raw.index)
train_raw.sample(5)   

#train_raw['ord_5'] = pd.Series([ord_five(x) for x in train_raw.ord_5], index=train_raw.index)
#train_raw['ord_5_plus'] = pd.Series([x for x in train_raw.ord_5], index=train_raw.index)

k-fold is used to determine the accuracy of the model with a greater degree of confidence. I reused the code for this function from a previous project: https://github.com/Neil-Kloper/u2n6lk8eWLzVfJ3M/blob/main/Apziva_eval_problem_Final.ipynb

The output from this is a dataframe that stores the average accuracy for the models, as well as the standard deviation of accuracy between folds.

In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics

kf = KFold(n_splits=6)

cols = ['Model', 'Mean train score', 'Train standard deviation', 'Mean test score', 'Test standard deviation']
model_performance = pd.DataFrame(columns=cols)

def kfold_eval_model(model, X_, metric=metrics.accuracy_score, update_df=True, printing=True):
    """simple function to test out a model for this problem
    statement in a single line of code,
    prints out the f1 score for training and testing data
    based on results from an n-fold cross validation
    saves results to a dataframe.
    will round predictions for regression to use for f1 evaluation.
    Takes an sklearn model as the first argument,
    second argument determines if the performance dataframe is updated"""
    global model_performance
    train_f1 = np.array([], dtype='float')
    test_f1 = np.array([], dtype='float')
    for train_index, test_index in kf.split(X_):
#         print(train_index)
        model.fit(X_.iloc[train_index], y.iloc[train_index])
        score_1 = metric(y.iloc[train_index], abs((model.predict(X_.iloc[train_index])).round()))
        score_2 =  metric(y.iloc[test_index], abs((model.predict(X_.iloc[test_index])).round()))
        train_f1 = np.append(train_f1, score_1)
        test_f1 = np.append(test_f1, score_2)
        train_stats = (train_f1.mean(), train_f1.std())
        test_stats = (test_f1.mean(), test_f1.std())
    if printing:
        print(model.__class__.__name__)
        print('Average train', metric.__name__, 'across', kf.get_n_splits(X),' folds:', train_stats[0], 'with a standard diviation of',train_stats[1])    
        print('Average test', metric.__name__, 'across', kf.get_n_splits(X),' folds:', test_stats[0], 'with a standard diviation of', test_stats[1])
    if update_df:
        s = np.array([[model.__class__.__name__, train_stats[0],  train_stats[1], test_stats[0],  test_stats[1]]])
        s = pd.DataFrame(s)
        s.columns = cols
        model_performance = model_performance.append(s)

I will use polynomial feature extraction to generate additional features so that the added features are not wholely without value. I will run through multiple iterations of multiple features and max depths for the decision trees.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pfe = PolynomialFeatures(degree=2)

y = train_raw.target
X = train_raw.drop('target', axis=1)
X_2 = pd.DataFrame(pfe.fit_transform(X))
X_2 = X.merge(X_2, right_index=True, left_index=True)

Here I test the Decision Trees performance given a max depth of 5

In [ ]:
from sklearn import tree
model_performance = pd.DataFrame(columns=cols)
dt = tree.DecisionTreeClassifier(max_depth=5) 
for num in range(20, 210, 10):
    dt.__class__.__name__ = str(num) + ' features'
    kfold_eval_model(dt, X_2[range(num)], printing=False) 
display(model_performance)

Next, I will repeat the same expirement, this time using 15 for the max depth.

In [ ]:
from sklearn import tree
model_performance = pd.DataFrame(columns=cols)
dt = tree.DecisionTreeClassifier(max_depth=15) 
for num in range(20, 210, 10):
    dt.__class__.__name__ = str(num) + ' features'
    kfold_eval_model(dt, X_2[range(num)], printing=False) 
display(model_performance)

Finally I will repeat the test with a max depth of 30

In [ ]:
from sklearn import tree
model_performance = pd.DataFrame(columns=cols)
dt = tree.DecisionTreeClassifier(max_depth=30) 
for num in range(20, 210, 10):
    dt.__class__.__name__ = str(num) + ' features'
    kfold_eval_model(dt, X_2[range(num)], printing=False) 
display(model_performance)